# <a name="0">Bias Mitigation for a Translation Service</a>

## <a name="sec1">Section 1: Evaluating a Large Language Model for Bias </a>
    
In this section, you evaluate a pre-trained large language model (LLM) for gender bias, and then implement bias mitigation strategies. You use the Dolly LLM to translate from one natural language (German) to another (English). You evaluate the translations for performance and bias with metrics from the Hugging Face Evaluate library. You then explore mitigating bias through prompting, a technique that can be used to make LLMs more fair. This section covers the following topics:

1. <a href="step1">Import libraries</a>
2. <a href="step2">Load an LLM</a>
3. <a href="step3">Translate a dataset from German to English</a>
4. <a href="step4">Evaluate for performance and bias</a>
5. <a href="step5">Use prompting</a>


Note: To avoid error messages due to missing code, work from top to bottom in this notebook, and do not skip sections.


### <a name="step1">Step 1: Import libraries</a>



First, install and import the necessary libraries, including the Hugging Face Transformers library and the Evaluate library.

Note: If you see an error alert about pip's dependency resolver, you can ignore it.

In [ ]:
%%capture

!export TOKENIZERS_PARALLELISM=false

!pip3 install -r requirements.txt
!pip install sagemaker --quiet --upgrade --force-reinstall

import warnings
warnings.filterwarnings('ignore')

Note: If you see a `ModuleNotFoundError` error alert, restart the kernel and start over.

In [ ]:
# Import libraries
import torch
from transformers import pipeline, AutoTokenizer
import pandas as pd
import tqdm
import evaluate
from rich import print

### <a name="step2">Step 2: Load an LLM</a>

Import the `dolly-v2-3B` pre-trained model from Databricks. The model is fine-tuned on `~15k records` generated by Databricks employees. This model has 2.8 billion parameters.

In [ ]:
# set seed for reproducible results
seed = 100
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Use a tokenizer suitable for Dolly-v2-3B
dolly_tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", padding_side = "left")

dolly_pipeline = pipeline(model = "databricks/dolly-v2-3b",
                          device_map = "auto",
                          torch_dtype = torch.float16,
                          trust_remote_code = True,
                          tokenizer = dolly_tokenizer)

### <a name="step3">Step 3: Translate a dataset from German to English</a>

Load the dataset used for this lab, the [Translated Wikipedia Biographies data](https://storage.googleapis.com/gresearch/translate-gender-challenge-sets/Readme.html) dataset from Google Research. Two .csv files are available: English to German and English to Spanish. You will use the English to German .csv file. Because the German translations were generated by professional human translators, you can translate them to English by using the model and compare the output to the English source text.

In [ ]:
wiki_bios_en_to_de = pd.read_csv("https://storage.googleapis.com/gresearch/translate-gender-challenge-sets/data/Translated%20Wikipedia%20Biographies%20-%20EN_DE.csv")

Now, take a look at the dataset. (You should familiarize yourself with the contents of a dataset that you use.) In this dataset, you see the language of the source text, which is English (`sourceLanguage`). You also see the target language for the translation (`targetLanguage`), the document ID (`documentID`), the string ID (`stringID`), the source text itself (`sourceText`), the professionally translated text (`translatedText`), the perceived gender as determined from the biography (`perceivedGender`), the subject of the biography (`entityName`), and the URL for the Wikipedia page (`sourceURL`).

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(wiki_bios_en_to_de.head())

To see how well the model translates, you will use the Dolly LLM to translate from German to English. Therefore, swap the column names so that `sourceLanguage` is German and `targetLanguage` is English, and so that `sourceText` is German and `translatedText` is English.

In [ ]:
wiki_bios_de_to_en = wiki_bios_en_to_de.rename(columns={"sourceLanguage": "targetLanguage", "targetLanguage": "sourceLanguage", "sourceText": "translatedText", "translatedText": "sourceText"})

with pd.option_context('display.max_colwidth', None):
    display(wiki_bios_de_to_en.head())

To help evaluate the Dolly LLM for gender bias in translations, divide the `wiki_bios_de_to_en` dataset by perceived gender of the subject of the biography. Then, randomly sample 100 observations from both the male and female subsets to avoid sampling bias by ensuring a balanced subset of the full dataset. Some observations, about bands or sports teams, have a neutral gender, so you can ignore these.

In [ ]:
print("Dataset size: " + str(wiki_bios_de_to_en.shape))

In [ ]:
male_bios = wiki_bios_de_to_en[wiki_bios_de_to_en.perceivedGender == "Male"]
female_bios = wiki_bios_de_to_en[wiki_bios_de_to_en.perceivedGender == "Female"]

print("Male Bios size: " + str(male_bios.shape))
print("Female Bios size: " + str(female_bios.shape))

male_sample = male_bios.sample(100, random_state=100)
female_sample = female_bios.sample(100, random_state=100)

print("Male Sample size: " + str(male_sample.shape))
print("Female Sample size: " + str(female_sample.shape))

You now have 100 text samples about males and 100 text samples about females. Provide these text samples to the model with the instruction to translate the text from German to English. Then, store these generations in a DataFrame and add them as a column to the `male_sample` DataFrame.

In [ ]:
male_generations = []
for row in tqdm.tqdm(range(len(male_sample))):
    source_text = male_sample.iloc[row]["sourceText"]
    # Create instruction to provide model
    cur_prompt_male = ("Translate \"%s\" from German to English." % (source_text))

    # Prompt model with instruction and text to translate
    generation = dolly_pipeline(cur_prompt_male)
    generated_text = generation[0]['generated_text']
    # Store translation
    male_generations.append(generated_text)

print('Generated '+ str(len(male_generations))+ ' male generations')

In [ ]:
# Add generations as a column to dataframe
male_sample["generatedText"] = male_generations

In [ ]:
female_generations = []
for row in tqdm.tqdm(range(len(female_sample))):
    source_text = female_sample.iloc[row]["sourceText"]
    cur_prompt_female = ("Translate \"%s\" from German to English." % (source_text))

    generation = dolly_pipeline(cur_prompt_female)
    generated_text = generation[0]['generated_text']
    female_generations.append(generated_text)

print('Generated '+ str(len(female_generations))+ ' female_generations')

In [ ]:
female_sample["generatedText"] = female_generations

In [ ]:
all_samples = pd.concat([male_sample, female_sample])

english = all_samples["translatedText"].values.tolist()
german = all_samples["sourceText"].values.tolist()
gender = all_samples["perceivedGender"].values.tolist()
generations = all_samples["generatedText"].values.tolist()

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(pd.DataFrame({'English from Human': english,'German from Human': german, 'English from LLM': generations, 'Perceived Gender': gender}, columns = ["English from Human", "German from Human", "English from LLM", "Perceived Gender"]))

### <a name="step4">Step 4: Evaluate for performance and bias</a>

Use two metrics, Bilingual Evaluation Understudy (BLEU) and Regard, to evaluate `Dolly-v2-3B` on the translations it produced. You can access these metrics through the Evaluate library from Hugging Face. This library has many other metrics that you can use to evaluate performance and fairness on various tasks. In this practice lab, you use BLEU to see the quality of the translations, and Regard to measure the language polarity of the translations for males and females. Remember that models should be both fair and perform well, so multiple metrics are needed for a holistic picture of how the model is performing.

Note that you are evaluating the `Dolly-v2-3B` model prior to any fine-tuning.

#### Bilingual Evaluation Understudy (BLEU)
[BLEU](https://huggingface.co/spaces/evaluate-metric/bleu) is used to measure the quality of text that has been translated from one natural language to another. This metric was introduced in ["BLEU: A Method for Automatic Evaluation of Machine Translation"](https://aclanthology.org/P02-1040.pdf). The measure is calculated by comparing the machine-generated translations to professional or reference translations, which are included in the dataset. To do so, the words in the reference text are compared to the model's output, and this is done for various n-grams, which are groups of one token (n=1), two tokens (n=2), three tokens (n=3), up to a maximum n-gram. This ensures that the score reflects both the similarity of the words themselves as well as their position in phrase. A score is determined for each text segment, and then this is aggregated over the dataset to determine the overall quality of the translations.

In [ ]:
# Load the BLEU metric from the evaluate library
bleu = evaluate.load("bleu")

The function `compute` compares the model's translations to the correct, professional translations included in the dataset. This gives a BLEU score, which ranges from 0 to 1, with values closer to 1 indicating greater similarity between the translations. In this case, that means the model is generating better translations. Start by computing the BLEU score for all 200 samples (both male and female). The `max_order` parameter corresponds to the maximum n-gram to use when computing the BLEU score.

In [ ]:
bleu.compute(predictions = all_samples["generatedText"].values.tolist(), references = all_samples["translatedText"].values.tolist(), max_order = 2)

Now, calculate the BLEU score for males and females separately.

In [ ]:
bleu.compute(predictions = male_sample["generatedText"].values.tolist(), references = male_sample["translatedText"].values.tolist(), max_order = 2)

In [ ]:
bleu.compute(predictions = female_sample["generatedText"].values.tolist(), references = female_sample["translatedText"].values.tolist(), max_order = 2)

This is a reasonable performance given that this model was not trained specifically for translations. The performance could be improved by fine-tuning specifically for the translation task.

#### Regard
[Regard](https://huggingface.co/spaces/evaluate-measurement/regard) measures language polarity and social perceptions towards a demographic. 

You are interested in the difference in Regard scores for male and female generations. To calculate this, input `male_generations` and `female_generations`. The output gives the difference in Regard scores for neutral, positive, negative, and other statements when comparing male to female. Adding `aggregation = "average"` gives the average score for each sentiment for each group, and adding `aggregation = "maximum"` gives the maximum Regard score for each group.

In [ ]:
# Load the Regard metric from evaluate
regard = evaluate.load("regard", "compare")

In [ ]:
regard.compute(data = male_generations, references = female_generations, aggregation = "average")

The first set of scores corresponds to `male_generations` and the second corresponds to `female_generations`. Observe the differences in neutral, positive, negative, and other. What does this indicate about the model's generations about males and females?

### <a name="step5">Step 5: Use prompting</a>

When interacting with an LLM, you can use prompting strategies to produce content that more closely aligns with fairness goals. This method a frugal because it does not require additional data or training to produce fairer responses. However, this method places responsibility on the user. 

In [ ]:
dolly_pipeline("""Translate from German to English and continue: "Casey studiert derzeit um eine Mathematiklehrkraft zu werden wegen".""")

In [ ]:
dolly_pipeline("""Translate from German to English and continue in a gender inclusive way: "Casey studiert derzeit um eine Mathematiklehrkraft zu werden wegen".""")

Observe that the translation differs if the additional goal of a gender inclusive translation is provided to the model.

## <a name="sec2">Section 2: Fine-Tuning the Dolly-3B Model To Use Inclusive Pronouns</a>

+ LLMs should learn how to generate inclusive pronouns.
+ You can fine-tune your model to accomplish it.
+ In this section, you fine-tune the Dolly-3B model to incorporate this knowledge by using data generated for the `fae` pronoun from a list of inclusive pronouns. The section covers the following topics:

6. <a href="step6">Prepare the training dataset</a>
7. <a href="step7">Load a pre-trained LLM</a>
8. <a href="step8">Define the trainer and fine-tune the LLM</a>
9. <a href="step9">Deploy the fine-tuned model</a>
10. <a href="step10">Test the deployed inference</a>

Run the following code block to import the necessary libraries, including the Hugging Face Transformers library and the PyTorch library (a dependency for transformers).

In [ ]:
%%capture

import os
import numpy as np
import pandas as pd
from typing import Any, Dict, List, Tuple, Union
from datasets import Dataset, load_dataset, disable_caching
disable_caching() ## disable huggingface cache

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import TextDataset

import torch
from torch.utils.data import Dataset, random_split
from transformers import TrainingArguments, Trainer
import accelerate
import bitsandbytes

from IPython.display import Markdown

!export TOKENIZERS_PARALLELISM=false

import warnings
warnings.filterwarnings('ignore')

### <a name="step6">Step 6: Prepare the training dataset</a>

Load and view the dataset. To reduce the runtime, you will use a smaller version of the original dataset.

In [ ]:
sagemaker_dataset = load_dataset("csv", 
                                    data_files='data/cda_fae_faer_faer_faerself.csv')['train']
sagemaker_dataset

In [ ]:
sagemaker_dataset[0]

#### <a name="step3">Step 6.1: Prepare the prompt</a>
To fine-tune the LLM, you must decorate the instruction dataset with a PROMPT, such as the following.

In [ ]:
from utils.helpers import INTRO_BLURB, INSTRUCTION_KEY, RESPONSE_KEY, END_KEY, RESPONSE_KEY_NL, DEFAULT_SEED, PROMPT
'''
PROMPT = """{intro}
            {instruction_key}
            {instruction}
            {response_key}
            {response}
            {end_key}"""
'''
Markdown(PROMPT)

Now, feed the PROMPT to the dataset through the following function, named `_add_text`. The function takes a record as input. The function first checks to ensure that both the instruction and response fields have values. If either of them is empty, the function raises a ValueError with a corresponding error alert. If both fields have values, the function creates a new "text" field in the record, formatting it by using the given PROMPT.

In [ ]:
def _add_text(rec):
    instruction = rec["instruction"]
    response = rec["response"]

    if not instruction:
        raise ValueError(f"Expected an instruction in: {rec}")

    if not response:
        raise ValueError(f"Expected a response in: {rec}")

    rec["text"] = PROMPT.format(
        instruction=instruction, response=response)

    return rec

Apply the mapping function with `.map`, and look at the format after mapping:

In [ ]:
sagemaker_dataset = sagemaker_dataset.map(_add_text)
sagemaker_dataset[0]

Use `Markdown` to neatly display the text with PROMPT.

In [ ]:
Markdown(sagemaker_dataset[0]['text'])

### <a name="step7">Step 7: Load a pre-trained LLM</a>


To load a pre-trained model, initialize a tokenizer and a base model by using the `databricks/dolly-v2-3b` model from the Hugging Face Transformers library. The tokenizer converts raw text into tokens, and the base model generates text based on a given prompt. By following the previous instructions, you can correctly instantiate these components and use their functionality in your code.


The `AutoTokenizer.from_pretrained()` function is used to instantiate the tokenizer. 
- `padding_side="left"` specifies the side of the sequences where padding tokens are added. In this case, padding tokens are added to the left side of each sequence. 
- `eos_token` is a special token representing the end of a sequence. By assigning it to `pad_token`, any padding tokens added during tokenization are considered as end-of-sequence tokens. This can be useful when generating text using the model, because it indicates when to stop generating text after encountering padding tokens.
- `tokenizer.add_special_tokens...` adds three additional special tokens to the tokenizer's vocabulary. These tokens likely serve specific purposes in the application using the tokenizer. For example, the tokens can be used to mark the end of an input, an instruction, or a response in a dialogue system.

After running, the `tokenizer` object is initialized and is ready to use for tokenizing text.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", 
                                          padding_side="left")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"additional_special_tokens": 
                              [END_KEY, INSTRUCTION_KEY, RESPONSE_KEY_NL]})

Pre-trained models generate text based on a given prompt (check the model limitations and preferred formats for prompting).
Now, initialize and download a base model using the `AutoModelForCausalLM` class provided by the Transformers library.

Different model classes are available in the Transformers library. CausalLMs are models that generate text for a given prompt.

Use the `AutoModelForCausalLM.from_pretrained()` function to instantiate the base model.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "databricks/dolly-v2-3b",
    # use_cache=False,
    device_map="auto", #"balanced",
    torch_dtype=torch.float16,
    load_in_8bit=True,
)

#### <a name="#step7.1">Step 7.1: Prepare the model for training</a>
Some preprocessing must be done before training such an int8 model using PEFT. Therefore, import a utility function, `prepare_model_for_int8_training`, that will do the following:

- Cast all the non `int8` modules to full precision (fp32) for stability.
- Add a forward_hook to the input embedding layer to enable gradient computation of the input hidden states.
- Enable gradient checkpointing for more memory-efficient training.

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Use the `preprocess_batch` function to preprocess the "text" field of the batch, applying tokenization, truncation, and other relevant operations based on the specified maximum length. The function takes a batch of data, a tokenizer, and a maximum length as input. 

Refer to `utils/helpers.py` file for more details.

In [ ]:
from functools import partial
from utils.helpers import mlu_preprocess_batch

MAX_LENGTH = 256
_preprocessing_function = partial(mlu_preprocess_batch, max_length=MAX_LENGTH, tokenizer=tokenizer)

Next, apply the preprocessing function to each batch in the dataset, modifying the "text" field accordingly. The map operation is performed in a batched manner and the "instruction", "response", and "text" columns are removed from the dataset. Finally, `processed_dataset` is created by filtering `sagemaker_dataset` based on the length of the "input_ids" field, ensuring that the length is less than the specified `MAX_LENGTH`.

In [ ]:
encoded_sagemaker_dataset = sagemaker_dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "response", "text"],
)

processed_dataset = encoded_sagemaker_dataset.filter(lambda rec: len(rec["input_ids"]) < MAX_LENGTH)

Split the dataset into `train` and `test` for evaluation.

In [ ]:
split_dataset = processed_dataset.train_test_split(test_size=14, seed=0)
split_dataset

### <a name="step8">Step 8: Define the trainer and fine-tune the LLM</a>

To efficiently fine-tune a model, you will use [LoRA: Low-Rank Adaptation](https://arxiv.org/abs/2106.09685). LoRA, freezes the pre-trained model weights and injects trainable rank decomposition matrices into each layer of the Transformer architecture, greatly reducing the number of trainable parameters for downstream tasks. Compared to GPT-3 175B, fine-tuned with Adam, LoRA can reduce the number of trainable parameters by 10,000 times and the GPU memory requirement by 3 times. 


#### <a name="#step8.1">Step 8.1: Define the `LoraConfig` and load the LoRA model</a> 

You will use the build LoRA class, `LoraConfig`, from [huggingface PEFT: State-of-the-art Parameter-Efficient Fine-Tuning](https://github.com/huggingface/peft). Within `LoraConfig`, specify the following parameters:

- `r`, the dimension of the low-rank matrices
- `lora_alpha`, the scaling factor for the low-rank matrices
- `lora_dropout`, the dropout probability of the LoRA layers
- `task_type`, allows prompt tuning for different tasks. In our case, causal language modeling

For more information about all available parameters, see Tuners on the Hugging Face PEFT page at https://huggingface.co/docs/peft/package_reference/tuners.

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

MICRO_BATCH_SIZE = 8  
BATCH_SIZE = 64
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LORA_R = 256 # 512
LORA_ALPHA = 512 # 1024
LORA_DROPOUT = 0.05

# Define LoRA Config
lora_config = LoraConfig(
                 r=LORA_R,
                 lora_alpha=LORA_ALPHA,
                 lora_dropout=LORA_DROPOUT,
                 bias="none",
                 task_type="CAUSAL_LM"
)

Use the `get_peft_model` function to initialize the model with the LoRA framework, configuring the model based on the provided `lora_config` settings. The model can then incorporate the benefits and capabilities of the LoRA optimization approach.

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

As you can see above, LoRA-only trainable parameters are only about three percent of the full weights. Very efficient!

#### <a name="#step8.2">Step 8.2: Define the data collator</a>

DataCollator is a Hugging Face transformers function that takes a list of samples from a dataset and collates them into a batch, as a dictionary of PyTorch tensors.

Use `DataCollatorForCompletionOnlyLM`, which extends the functionality of the base `DataCollatorForLanguageModeling` class from the Transformers library. This custom collator is designed to handle examples where a prompt is followed by a response in the input text, and then modify the labels accordingly.

Refer to `utils/helpers.py` for the implementation.

In [ ]:
from utils.helpers import MLUDataCollatorForCompletionOnlyLM

data_collator = MLUDataCollatorForCompletionOnlyLM(
        tokenizer=tokenizer, mlm=False, return_tensors="pt", pad_to_multiple_of=8
)

#### <a name="#step8.3">Step 8.3: Define the trainer</a>

To fine-tune the LLM, you must define a `Trainer`. First, define some training arguments.

Find more information about the `Trainer`class, see Trainer on the Hugging Face Transformers page at https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/trainer.

In [ ]:
EPOCHS = 10
LEARNING_RATE = 1e-4  
MODEL_SAVE_FOLDER_NAME = "dolly-3b-lora"

training_args = TrainingArguments(
                    output_dir=MODEL_SAVE_FOLDER_NAME,
                    fp16=True,
                    per_device_train_batch_size=1,
                    per_device_eval_batch_size=1,
                    learning_rate=LEARNING_RATE,
                    num_train_epochs=EPOCHS,
                    logging_strategy="steps",
                    logging_steps=100,
                    evaluation_strategy="steps",
                    eval_steps=100, 
                    save_strategy="steps",
                    save_steps=20000,
                    save_total_limit=10,
)

Now is when the magic happens! Initialize the trainer with the defined model, tokenizer, training arguments, data collator, and the train/eval datasets. 

<div class="alert alert-block alert-warning">
<b>Warning:</b> <br/>
The training might take about 10 minutes to run with the <code>fae/faer/faerself</code> data from <code>cda_fae_faer_faer_faerself.csv</code>.
</div>

In [ ]:
trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=split_dataset['train'],
        eval_dataset=split_dataset["test"],
        data_collator=data_collator,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

#### <a name="#step8.4">Step 8.4: Save the fine-tuned model</a>


After the training is finished, you can save the model to a directory by using the [`transformers.PreTrainedModel.save_pretrained`] function. 
This function saves only the incremental PEFT weights (adapter_model.bin) that were trained, meaning the model is very efficient to store, transfer, and load.

In [ ]:
trainer.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)

In [ ]:
trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

Save the tokenizer along with the trained model.

In [ ]:
tokenizer.save_pretrained(MODEL_SAVE_FOLDER_NAME)

### <a name="step9">Step 9: Deploy the fine-tuned model</a>

#### <a name="step9title">Overview of deployment parameters</a>

To deploy using the Amazon SageMaker Python SDK with the DJL, you must instantiate `Model` class with the following parameters:
```{python}
model = Model(
    image_uri,
    model_data=...,
    predictor_cls=...,
    role=aws_role
)
```
- `image_uri`: The Docker image URI representing the deep learning framework and version to be used.
- `model_data`: The location of the fine-tuned LLM model artifact in an Amazon Simple Storage Service (Amazon S3) bucket. It specifies the path to the TAR GZ file containing the model's parameters, architecture, and any necessary artifacts.
- `predictor_cls`: This is a "JSON in JSON out" predictor only, nothing DJL related. For more information, see sagemaker.djl_inference.DJLPredictor at https://sagemaker.readthedocs.io/en/stable/frameworks/djl/sagemaker.djl_inference.html#djlpredictor.
- `role`: The AWS Identity and Access Management (IAM) role ARN that provides necessary permissions to access resources, such as the S3 bucket containing the model data.

#### <a name="step9.1">Step 9.1: Instantiate SageMaker parameters</a>

Initialize a SageMaker session and retrieve information related to the AWS environment, such as SageMaker role and AWS region. You also specify the image URI for a specific version of the "djl-deepspeed" framework using the SageMaker session's Region. The image URI is a unique identifier for a specific Docker container image that can be used in various AWS services, such as Amazon SageMaker or Amazon Elastic Container Registry (Amazon ECR).

In [ ]:
import boto3
import json
import sagemaker.djl_inference
from sagemaker.session import Session
from sagemaker import image_uris
from sagemaker import Model

sagemaker_session = Session()
print("sagemaker_session: ", sagemaker_session)

aws_role = sagemaker_session.get_caller_identity_arn()
print("aws_role: ", aws_role)

aws_region = boto3.Session().region_name
print("aws_region: ", aws_region)

image_uri = image_uris.retrieve(framework="djl-deepspeed",
                                version="0.22.1",
                                region=sagemaker_session._region_name)
print("image_uri: ", image_uri)

#### <a name="step9.2">Step 9.2: Create the model artifact</a> ###

To upload the model artifact in the S3 bucket, you must create a TAR GZ file containing the model's parameters. First, create a directory named `lora_model` and a subdirectory named `dolly-3b-lora`. The "-p" option ensures that the command creates any intermediate directories if they don't exist. Then, copy the lora checkpoints `adapter_model.bin` and `adapter_config.json` to `dolly-3b-lora`. The base Dolly model will be downloaded at runtime from the Hugging Face hub.

In [ ]:
%%bash
rm -rf lora_model
mkdir -p lora_model
mkdir -p lora_model/dolly-3b-lora
cp dolly-3b-lora/adapter_config.json lora_model/dolly-3b-lora/
cp dolly-3b-lora/adapter_model.bin lora_model/dolly-3b-lora/

Next, set the [DJL Serving configuration options](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html) in `serving.properties`. Using the Jupyter `%%writefile` magic command, you can write the following content to a file named "lora_model/serving.properties".
- `engine=Python`: This line specifies the engine used for serving.
- `option.entryPoint=model.py`: This line specifies the entry point for the serving process, which is set to "model.py". 
- `option.adapter_checkpoint=dolly-3b-lora`: This line sets the checkpoint for the adapter to "dolly-3b-lora". A checkpoint typically represents the saved state of a model or its parameters.
- `option.adapter_name=dolly-lora`: This line sets the name of the adapter to "dolly-lora", a component that helps interface between the model and the serving infrastructure.

In [ ]:
%%writefile lora_model/serving.properties
engine=Python
option.entryPoint=model.py
option.adapter_checkpoint=dolly-3b-lora
option.adapter_name=dolly-lora

Another file you need in the model artifact is the environment requirement file. Create a file named `lora_model/requirements.txt`, and write a list of Python package requirements, typically used with package managers such as `pip`.

In [ ]:
%%writefile lora_model/requirements.txt
transformers==4.27.4
accelerate>=0.24.1,<1
peft

#### <a name="step9.3">Step 9.3: Create the inference script</a>

Similar to the fine-tuning notebook, a custom pipeline `InstructionTextGenerationPipeline` is defined. The code is provided in `utils/deployment_model.py`. 

You save these inference functions to `lora_model/model.py`.

In [ ]:
%%bash
cp utils/deployment_model.py lora_model/model.py

#### <a name="step9.4">Step 9.4: Upload the model artifact to Amazon S3</a>

Create a compressed tarball archive of the "lora_model" directory, and then save it as "lora_model.tar.gz".

In [ ]:
%%bash
tar -cvzf lora_model.tar.gz lora_model/

Upload the "lora_model.tar.gz" file to the specified S3 bucket.

In [ ]:
import boto3
import json
import sagemaker.djl_inference
from sagemaker.session import Session
from sagemaker import image_uris
from sagemaker import Model

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

s3 = boto3.resource('s3')

# Get the name of the bucket with prefix lab-code
for bucket in s3.buckets.all():
    if bucket.name.startswith('artifact'):
        mybucket = bucket.name
        print(mybucket)
    
response = s3_client.upload_file("lora_model.tar.gz", mybucket, "lora_model.tar.gz")

#### <a name="step9.5">Step 9.5: Deploy the model</a> ###

Now, it's the time to deploy the fine-tuned LLM by using the SageMaker Python SDK. The SageMaker Python SDK `Model` class is instantiated with the following parameters:

- `image_uri`: The Docker image URI representing the deep learning framework and version to be used.
- `model_data`: The location of the fine-tuned LLM model artifact in an S3 bucket. It specifies the path to the TAR GZ file containing the model's parameters, architecture, and any necessary artifacts.
- `predictor_cls`: This is a "JSON in JSON out" predictor only, nothing DJL related. For more information, see sagemaker.djl_inference.DJLPredictor at https://sagemaker.readthedocs.io/en/stable/frameworks/djl/sagemaker.djl_inference.html#djlpredictor.
- `role`: The AWS Identity and Access Management (IAM) role ARN that provides necessary permissions to access resources, such as the S3 bucket containing the model data.

In [ ]:
model_data="s3://{}/lora_model.tar.gz".format(mybucket)

model = Model(image_uri=image_uri,
            model_data=model_data,
            predictor_cls=sagemaker.djl_inference.DJLPredictor,
            role=aws_role)

Note: The deployment should be completed within 10 minutes. If it takes longer than that, your endpoint might have failed.

In [ ]:
%%time
predictor = model.deploy(1, "ml.g4dn.2xlarge")